In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import sep_vvkernels as svvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
from LBFGS import FullBatchLBFGS
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N

def g_theta(sample_size, D):
    loc = np.random.random_sample((sample_size,D))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
train_y = torch.zeros([sample_size, N])




for i in range(sample_size):
    train_y[i] = Tensor(vf(train_x[i])) + torch.randn(Tensor(vf(train_x[i])).size()) *  math.sqrt(0.04)
    

In [3]:
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) + torch.randn(Tensor(vf(x[i])).size()) * math.sqrt(0.04)
    return out

In [4]:
def stopping_criteria(tol_vector, f_target, lower_bound, upper_bound):
    lower_tol_vector = f_target - tol_vector
    upper_tol_vector = f_target + tol_vector
    SUCCESS = True
    FAILURE = False
    for i in range(f_target.shape[0]):
            if (lower_bound[i] < lower_tol_vector[i]) or  (upper_bound[i] > upper_tol_vector[i]):
                SUCCESS = False  
            if ((lower_bound[i] > upper_tol_vector[i]) or  (upper_bound[i] < lower_tol_vector[i])):
                FAILURE = True
    return SUCCESS, FAILURE

In [5]:

x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,num_base_kernels):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
        base_kernels = []
        for i in range(num_base_kernels):
            base_kernels.append(vvk_rbf.vvkRBFKernel())


        self.covar_module = svvk.SepTensorProductKernel(base_kernels,num_tasks = 2)

#\         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
       # self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    


In [6]:
# ##hyperparameters optimization###
# def hyper_opti(g_theta1, agg_data, training_iter):
#     likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

#     model = MultitaskGPModel(g_theta1, agg_data, likelihood,3)
#     model.train()
#     likelihood.train()
    
    

#     optimizer = torch.optim.Adam(model.parameters(),  lr=0.07)  # Includes GaussianLikelihood parameters
#     #mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
#     for i in range(training_iter):
#         optimizer.zero_grad()
#         output = model(g_theta1)
#         output_ll = likelihood(output)

#         loss = -likelihood.get_mll(agg_data,output_ll)
#         #loss = -mll(output, agg_data)
#         loss.backward(retain_graph=True)

#         print('Iter %d/%d - Loss hyperparam: %.3f' % (i + 1, training_iter, loss.item()))
#         optimizer.step()

#     return model, likelihood


In [7]:
def hyper_opti(g_theta1, agg_data, training_iter):
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

    model = MultitaskGPModel(g_theta1, agg_data, likelihood,3)
    model.train()
    likelihood.train()
    optimizer = FullBatchLBFGS(model.parameters(), lr=1.)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    def closure():
        
        optimizer.zero_grad()
        output = model(g_theta1)
        #output_ll = likelihood(output)

        #loss = -likelihood.get_mll(agg_data,output_ll)
        loss = -mll(output, agg_data)
        print('Loss gp: %.3f' % ( loss))
        
        return loss
    
    loss = closure()
    loss.backward()

    for i in range(training_iter):
        options = {'closure': closure, 'current_loss': loss, 'max_ls': 20}
        loss, _, _, _, _, _, _, fail = optimizer.step(options)

        if fail:
            print('Convergence reached!')
            break

 
    
 


    return model, likelihood

In [8]:
class param_opti(nn.Module):
    def __init__(self, sample):
        super(param_opti, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        self.g_theta2 = nn.Parameter(Tensor(sample))
        
    def forward(self):
        
        return (self.g_theta2)

In [9]:
def conduct_param_opti(x,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_iter):
    model.eval()
    likelihood.eval()
    _par = param_opti(loc_sample)
    
    def closure():
        optimizer.zero_grad()
        g_theta2 = _par.forward()

        loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
        loss1 = -1 * loss1
        
        return loss1
    
    
    optimizer = torch.optim.LBFGS(_par.parameters(), lr=1., history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)
    
    g_theta2 = _par.forward()

    loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
    loss1 = -1 * loss1
    
    print('Loss theta: %.3f' % ( loss1))
    
#     optimizer = torch.optim.Adam(_par.parameters(), lr=0.007)

# #     for j in range(training_iter):
# #         optimizer.zero_grad()
# #         g_theta2 = _par.forward()

# #         loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
# #         loss1 = -1 * loss1
# #         loss1.backward(retain_graph=True)
# #         print('Iter %d/%d - Loss theta2: %.3f' % (j + 1, training_iter, loss1.item()))
# #         optimizer.step()
          
    return loss1, g_theta2, lower_bound, upper_bound

In [10]:
class design_opti(nn.Module):
    def __init__(self, _x):
        super(design_opti, self).__init__()
#         a = _x[0,0]
#         b = _x[0,1]
#         self.x_design_1 = nn.Parameter(a)
#         self.x_design_2 = nn.Parameter(b)
        self.x_design = nn.Parameter(_x)
    def forward(self):
#         x_design = torch.zeros(1,2)
#         x_design[0,0] = self.x_design_1
#         x_design[0,1] = self.x_design_2
        return (self.x_design)

In [11]:
def conduct_design_opti(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter, training_param_iter, lr_new):
    design = design_opti(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    
    def closure():
        optimizer.zero_grad()
        
        loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter)
        loss2.backward(retain_graph=True)
#         print(x_d)
#         print(lower_bound)
#         print(upper_bound)
       
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=lr_new, history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter) ##takethis out
    print('Loss design: %.3f' % ( loss2))
    print(optimizer.state)
    return x_d, g_theta2_out, lower_bound, upper_bound
    
    

In [12]:
iter_hp = 90
iter_design = 40 
iter_param = 50


f_target = Tensor(vf.tgt_vec) 
f_target = f_target.reshape(f_target.shape[0],1) 
tol_vector = 0.005 * torch.ones(f_target.shape)


loc_size = 10
loc_sample = np.random.random_sample((loc_size,2))
g_theta2_vec = (Tensor(loc_sample)).flatten()

g_theta1 = x_train
agg_data = y_train.flatten()


x0 = Tensor(np.array([0.5,0.7])) 
x0 = x0.reshape(1,2)
x00 = x0 
vec_x = x00 


lr_new = 1.

SUCCESS = False 
FAILURE = False 
iter = 0 
tol = 0.009 
while(SUCCESS == False and iter < 50):
    print('START HYPERPARAMETERS optimization')
    model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp)

    print('END HYPERPARAMETERS optimization')
    print(iter)
    
    x0_new,g_theta2,lower_bound, upper_bound = conduct_design_opti(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design,iter_param, lr_new)
    g_theta2_vec = torch.cat([g_theta2_vec, g_theta2.flatten()], 0)
    print(torch.norm(upper_bound - tol_vector))
#     if (torch.norm(upper_bound - tol_vector) <= 0.1 ):
#         print('bkhjghf')
#         lr_new = lr_new * 0.1
    print(lower_bound)
    print(upper_bound)
    print(f_target-tol_vector)
    print(f_target+tol_vector)
    loc_sample = np.random.random_sample((loc_size,2))
    x0 = x0_new #Tensor(np.random.random_sample((1,2))) #x0_new
    vec_x = torch.cat([vec_x, x0_new])
    g_theta2_detach = g_theta2.detach()
    new_data = vfield_(g_theta2_detach)
    agg_data = torch.cat([agg_data, new_data.flatten()], 0)
    g_theta1= torch.cat([g_theta1, g_theta2], 0)

    SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)
    iter = iter + 1
    print(x0_new)

START HYPERPARAMETERS optimization
Loss gp: 0.898
Loss gp: 0.796
Loss gp: 0.690
Loss gp: 0.473
Loss gp: 0.102
Loss gp: 27.755
Loss gp: 0.039
Loss gp: 0.010
Loss gp: 0.003
Loss gp: -0.011
Loss gp: -0.021
Loss gp: -0.036
Loss gp: -0.039
Loss gp: -0.040
Loss gp: -0.042
Loss gp: -0.047
Loss gp: -0.052
Loss gp: -0.051
Loss gp: -0.053
Loss gp: -0.054
Loss gp: -0.056
Loss gp: -0.058
Loss gp: -0.058
Loss gp: -0.060
Loss gp: -0.060
Loss gp: -0.061
Loss gp: -0.061
Loss gp: -0.062
Loss gp: -0.061
Loss gp: -0.062
Loss gp: -0.062
Loss gp: -0.062
Loss gp: -0.063
Loss gp: -0.064
Loss gp: -0.062
Loss gp: -0.064
Loss gp: -0.060
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Loss gp: -0.064
Convergence reached!
END HYPERPARAMETERS opti

Loss theta: -6.182
Loss theta: -6.017
Loss theta: -6.232
Loss theta: -6.156
Loss theta: -6.205
Loss theta: -6.191
Loss theta: -6.232
Loss theta: -6.232
Loss design: -6.232
defaultdict(<class 'dict'>, {Parameter containing:
tensor([[0.2319, 0.0822]], requires_grad=True): {'func_evals': 11, 'n_iter': 4, 'al': [tensor(0.0239), tensor(0.3055), tensor(-0.1744), None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'd': tensor([-0.0008,  0.0007]), 't': 0, 'old_dirs': [tens

Loss gp: 0.747
Loss gp: 0.636
Loss gp: 0.410
Loss gp: 0.033
Loss gp: 9.273
Loss gp: -0.023
Loss gp: -0.032
Loss gp: -0.034
Loss gp: -0.040
Loss gp: -0.044
Loss gp: -0.052
Loss gp: -0.056
Loss gp: -0.057
Loss gp: -0.064
Loss gp: -0.068
Loss gp: -0.069
Loss gp: -0.071
Loss gp: -0.072
Loss gp: -0.072
Loss gp: -0.073
Loss gp: -0.073
Loss gp: -0.072
Loss gp: -0.073
Loss gp: -0.074
Loss gp: -0.074
Loss gp: -0.075
Loss gp: -0.075
Loss gp: -0.076
Loss gp: -0.078
Loss gp: -0.080
Loss gp: -0.067
Loss gp: -0.080
Loss gp: -0.081
Loss gp: -0.082
Loss gp: -0.001
Loss gp: -0.081
Loss gp: -0.083
Loss gp: -0.076
Loss gp: -0.083
Loss gp: -0.081
Loss gp: -0.084
Loss gp: -0.080
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.084
Loss gp: -0.0

Loss gp: 0.393
Loss gp: 0.008
Loss gp: 11.124
Loss gp: -0.048
Loss gp: -0.060
Loss gp: -0.062
Loss gp: -0.067
Loss gp: -0.072
Loss gp: -0.079
Loss gp: -0.084
Loss gp: -0.088
Loss gp: -0.091
Loss gp: -0.093
Loss gp: -0.094
Loss gp: -0.096
Loss gp: -0.097
Loss gp: -0.099
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.101
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.101
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Loss gp: -0.100
Convergence reached!
END HYPERPARAMETERS optimization
9
Loss theta: -5.675
Loss theta: 725.928
Loss theta: -5.702
Loss theta: -3.633
Loss theta: -5.746
Loss theta: -5.762
Loss theta: -5.792
Loss theta: -5.974
Loss theta: -5.993
Loss theta: -5.996
Loss theta: -5.984
Loss theta: -5.997
Loss theta: -5.986
Loss theta

Loss gp: -0.079
Loss gp: -0.086
Loss gp: -0.092
Loss gp: -0.093
Loss gp: -0.093
Loss gp: -0.094
Loss gp: -0.097
Loss gp: -0.099
Loss gp: -0.101
Loss gp: -0.102
Loss gp: -0.103
Loss gp: -0.105
Loss gp: -0.105
Loss gp: -0.105
Loss gp: -0.106
Loss gp: -0.106
Loss gp: -0.106
Loss gp: -0.106
Loss gp: -0.107
Loss gp: -0.106
Loss gp: -0.108
Loss gp: -0.107
Loss gp: -0.108
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Loss gp: -0.109
Convergence reached!
END HYPERPARAMETERS optimization
12
Loss theta: -6.862
Loss theta: -4.073
Loss theta: -5.296
Loss theta: -6.856
Loss theta: -6.887
Loss theta: -6.885
Loss theta: -6.892
Loss theta: -6.907
Loss theta: -6.916
Loss

...

Loss gp: 0.577
Loss gp: 0.336
Loss gp: -0.062
Loss gp: 6.680
Loss gp: -0.110
Loss gp: -0.117
Loss gp: -0.118
Loss gp: -0.120
Loss gp: -0.122
Loss gp: -0.126
Loss gp: -0.131
Loss gp: -0.132
Loss gp: -0.133
Loss gp: -0.133
Loss gp: -0.135
Loss gp: -0.135
Loss gp: -0.136
Loss gp: -0.137
Loss gp: -0.137
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.138
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.137
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.138
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Loss gp: -0.139
Convergence

Loss gp: 0.574
Loss gp: 0.332
Loss gp: -0.069
Loss gp: 7.461
Loss gp: -0.118
Loss gp: -0.126
Loss gp: -0.128
Loss gp: -0.129
Loss gp: -0.132
Loss gp: -0.137
Loss gp: -0.140
Loss gp: -0.140
Loss gp: -0.141
Loss gp: -0.142
Loss gp: -0.143
Loss gp: -0.145
Loss gp: -0.145
Loss gp: -0.145
Loss gp: -0.145
Loss gp: -0.146
Loss gp: -0.146
Loss gp: -0.146
Loss gp: -0.146
Loss gp: -0.147
Loss gp: -0.126
Loss gp: -0.146
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Loss gp: -0.147
Convergence reached!
END HYPERPARAMETERS optimization
39
Loss theta: -6.798
Loss theta: 1.337
Loss theta: -1.753
Loss theta: -6.816
Loss theta: -6.819
Loss

Loss gp: 0.570
Loss gp: 0.326
Loss gp: -0.076
Loss gp: 7.326
Loss gp: -0.125
Loss gp: -0.132
Loss gp: -0.134
Loss gp: -0.135
Loss gp: -0.138
Loss gp: -0.142
Loss gp: -0.145
Loss gp: -0.145
Loss gp: -0.145
Loss gp: -0.146
Loss gp: -0.147
Loss gp: -0.148
Loss gp: -0.150
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.150
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Convergence reached!
END HYPERPARAMETERS optimization
42
Loss theta: -7.011
Loss theta: 0.909
Loss theta: -3.677
Loss theta: -7.008
Loss theta: -7.027
Loss theta: -7.019
Loss theta: -7.022
Loss theta: -7.025
Loss theta: -7.027
Loss theta: -7.027
Loss design: -7.027
defaultdict(<class 'dict'>, {Parameter containing:
tensor([[0.0890, 0.1992]], requires_grad=True): {'func_evals':

Loss gp: -0.145
Loss gp: -0.146
Loss gp: -0.148
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.149
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Loss gp: -0.150
Convergence reached!
END HYPERPARAMETERS optimization
45
Loss theta: -7.073
Loss theta: 3349.622
Loss theta: -7.057
Loss theta: -7.064
Loss theta: -7.061
Loss theta: -7.071
Loss theta: -7.073
Loss theta: -7.073
Loss design: -7.073
defaultdict(<class 'dict'>, {Parameter containing:
tensor([[0.0915, 0.2012]], requires_grad=True): {'func_evals': 7, 'n_iter': 1,

Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.273
Loss theta: -7.241
Loss theta: -7.247
Loss theta: -7.283
Loss theta: -7.269
Loss theta: -7.283
Loss theta: -7.283
Loss design: -7.283
defaultdict(<class 'dict'>, {Parameter containing:
tensor([[0.1366, 0.1554]], requires_grad=True): {'func_evals': 34, 'n_iter': 5, 'al': [tensor(-0.0037), tensor(0.0020), tensor(3.1848), None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [13]:
#x0 = Tensor(np.array([0.1937, 0.1257]))
#x0 = Tensor(np.array([0.1885, 0.1038]))

x0 = Tensor(np.array([0.1566, 0.1873])) # lr = .2, samp_size = 5
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

[0.52541146 1.06348491]
tensor([[0.1566, 0.1873]])
tensor([[0.5375, 1.0537]], grad_fn=<ViewBackward>)


In [14]:
iter

50

In [15]:
res = 0
res = res + torch.ones(10,10)

In [16]:
print(g_theta2_vec.shape)

torch.Size([1020])


In [17]:
v2 = g_theta2_vec.reshape(70,2)
print(vec_x)

RuntimeError: shape '[70, 2]' is invalid for input of size 1020

In [ ]:
print(v2)
np.savetxt('g_theta2_01.txt', v2.detach().numpy())

In [ ]:

np.savetxt('vec_x_01.txt', vec_x.detach().numpy())